<a href="https://colab.research.google.com/github/palfans/HCL_GENAI_TRAINING/blob/main/house-price-prediction-deep-learning-3d-plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Input, Dense

dataset_url = "https://raw.githubusercontent.com/enuguru/aiandml/refs/heads/master/machine_learning_algorithms_using_frameworks/python_files/regression/house_price_prediction/home.csv"
data = pd.read_csv(dataset_url, encoding='utf-8-sig', header=None)
data.columns = ['sqft', 'bedrooms', 'price']

features = data[['sqft', 'bedrooms']]
target = data['price']

x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

scaler_x = StandardScaler()
x_train_scaled = scaler_x.fit_transform(x_train)
x_test_scaled = scaler_x.transform(x_test)

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

model = Sequential()
model.add(Input(shape=(2,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit(x_train_scaled, y_train_scaled, epochs=50, batch_size=5, verbose=1, validation_split=0.1)


sqft_range = np.linspace(features['sqft'].min(), features['sqft'].max(), 50)
bedrooms_range = np.linspace(features['bedrooms'].min(), features['bedrooms'].max(), 50)
sqft_grid, bedrooms_grid = np.meshgrid(sqft_range, bedrooms_range)
input_grid = np.c_[sqft_grid.ravel(), bedrooms_grid.ravel()]
input_grid_df = pd.DataFrame(input_grid, columns=['sqft', 'bedrooms'])
input_grid_scaled = scaler_x.transform(input_grid_df)
predicted_prices_scaled = model.predict(input_grid_scaled)
predicted_prices = scaler_y.inverse_transform(predicted_prices_scaled)

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

ax.plot_surface(sqft_grid, bedrooms_grid,
                predicted_prices.reshape(sqft_grid.shape),
                color='cyan', alpha=0.6, linewidth=0, antialiased=False)
ax.scatter(features['sqft'], features['bedrooms'], target, c='r', marker='o')

ax.set_xlabel('Sqft')
ax.set_ylabel('Bedrooms')
ax.set_zlabel('Price')
ax.set_title('3D House Price Prediction with Deep Learning')
plt.show()